# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Data Science
import pandas as pd
import numpy as np

# API Requests
from pprint import pprint
import requests
import json

# Data Viz
import hvplot.pandas
import matplotlib.pyplot as plt
import seaborn as sns

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
df = pd.read_csv("output_data/cities.csv", index_col="City_ID")
# Display sample data
df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
City_ID,,,,,,,,,
0,Lubang,13.8584,120.1240,79.36,85,100,17.60,PH,1732995926
1,Udachny,66.4167,112.4000,-29.16,100,96,3.06,RU,1732995927
2,Yar,58.2490,52.1026,30.07,94,98,9.22,RU,1732995928
3,Blackmans Bay,-43.0167,147.3167,60.46,96,100,7.16,AU,1732995929
4,Waitangi,-43.9535,-176.5597,53.82,77,13,11.99,NZ,1732995862


In [7]:
# Date Cleaning w/dates
df["Date"] = pd.to_datetime(df.Date * 1e9)
df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
City_ID,,,,,,,,,
0,Lubang,13.8584,120.1240,79.36,85,100,17.60,PH,2024-11-30 19:45:26
1,Udachny,66.4167,112.4000,-29.16,100,96,3.06,RU,2024-11-30 19:45:27
2,Yar,58.2490,52.1026,30.07,94,98,9.22,RU,2024-11-30 19:45:28
3,Blackmans Bay,-43.0167,147.3167,60.46,96,100,7.16,AU,2024-11-30 19:45:29
4,Waitangi,-43.9535,-176.5597,53.82,77,13,11.99,NZ,2024-11-30 19:44:22


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 584 entries, 0 to 583
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   City        584 non-null    object        
 1   Lat         584 non-null    float64       
 2   Lng         584 non-null    float64       
 3   Max Temp    584 non-null    float64       
 4   Humidity    584 non-null    int64         
 5   Cloudiness  584 non-null    int64         
 6   Wind Speed  584 non-null    float64       
 7   Country     582 non-null    object        
 8   Date        584 non-null    datetime64[ns]
dtypes: datetime64[ns](1), float64(4), int64(2), object(2)
memory usage: 45.6+ KB


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [11]:
%%capture --no-display

# Configure the map plot
map_plot = df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriNatGeo",
    frame_width = 700,
    frame_height = 500,
    color = "City",
    hover_cols=["City", "Country", "Max Temp"]  # Add 'city' to the tooltip
)

# Display the map plot
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Country,Max Temp)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [13]:
# Narrow down cities that fit criteria and drop any results with null values
min_temp = 70
max_temp = 80
max_wind = 10

# Drop any rows with null values
df2 = df.dropna()

mask = (df2["Max Temp"] >= min_temp) & (df2["Max Temp"] < max_temp) & (df2["Wind Speed"] < max_wind)
df2 = df2.loc[mask].reset_index(drop=True)

# Display sample data
df2

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Pathein,16.7833,94.7333,71.60,86,45,4.25,MM,2024-11-30 19:46:03
1,Bandar-e Lengeh,26.5579,54.8807,70.66,27,0,7.67,IR,2024-11-30 19:46:05
2,Koné,-21.0595,164.8658,77.68,67,1,3.53,NC,2024-11-30 19:46:06
3,Piti Village,13.4626,144.6933,79.00,92,0,5.75,GU,2024-11-30 19:46:10
4,Tazacorte,28.6290,-17.9293,71.15,56,0,8.01,ES,2024-11-30 19:46:16
...,...,...,...,...,...,...,...,...,...
93,Labuan,5.2767,115.2417,75.09,94,20,0.00,MY,2024-11-30 19:56:53
94,Ponnāni,10.7670,75.9252,78.51,84,100,7.87,IN,2024-11-30 19:56:54
95,Yeppoon,-23.1333,150.7333,74.46,86,16,2.53,AU,2024-11-30 19:56:57
96,Al Muwayh,22.4333,41.7583,70.39,43,0,7.27,SA,2024-11-30 19:57:00


### Step 3: Create a new DataFrame called `hotel_df`.

In [21]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity

hotel_df = pd.DataFrame(hotel_rows)
hotel_df

,city,country,latitude,longitude,address,name,distance,elevation,website
0,Pathein,MM,16.7833,94.7333,"Taan Taan Ta, Merchant Street, Pathein, 10011,...",Taan Taan Ta,333.0,None,None
1,Bandar-e Lengeh,IR,26.5579,54.8807,"Old owner do not like foreigners hotel, Bandar...",Old owner do not like foreigners hotel,571.0,None,None
2,Koné,NC,-21.0595,164.8658,"Pacifik Appartels, 213 Rue Auguste Henriot, 98...",Pacifik Appartels,411.0,None,https://www.pacifikappartels.nc/fr/index.html
3,Piti Village,GU,13.4626,144.6933,"The Cliff Hotel, Frank Javier Avenue, Agana He...",The Cliff Hotel,6182.0,None,None
4,Tazacorte,ES,28.6290,-17.9293,"App Leyma, Avenida Felipe Lorenzo, 8, 38770 Ta...",App Leyma,1250.0,None,None
...,...,...,...,...,...,...,...,...,...
93,Labuan,MY,5.2767,115.2417,"OYO 89344 Labuan Avenue Hotel, Jalan OKK Awang...",OYO 89344 Labuan Avenue Hotel,196.0,None,None
94,Ponnāni,IN,10.7670,75.9252,"Hotel Salkara, over bridge, Ponnani - 679586, ...",Hotel Salkara,653.0,None,None
95,Yeppoon,AU,-23.1333,150.7333,"Blossom's on Seaspray, Wattle Grove, Cooee Bay...",Blossom's on Seaspray,2198.0,None,None
96,Al Muwayh,SA,22.4333,41.7583,"فندق ومطعم الموية, Makkah Al-Mukarramah Road, ...",فندق ومطعم الموية,611.0,None,None


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [26]:
hotel_rows = []

for index, row in df2.iterrows():
    # from the Weather CSV
    city = row["City"]
    country = row["Country"]
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Set the parameters for the type of place
    categories = "accommodation.hotel"
    radius = 10000 # 10km
    
    # Set the parameters for the type of search
    filters = f"circle:{longitude},{latitude},{radius}"
    bias = f"proximity:{longitude},{latitude}"
    limit = 5
    
    # set up a parameters dictionary
    params = {
        "categories":categories,
        "limit":limit,
        "filter":filters,
        "bias":bias,
        "apiKey":geoapify_key    
    }
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Run request
    try:
        response = requests.get(base_url, params=params)
        # print(response.status_code)
        data = response.json()
        
        # Print the results
        results = data.get("features", [])
        
        # Resiliency/Error Handling
        if len(results) > 0:
            place = results[0]
    
            # normalize data
            address = place.get("properties", {}).get("formatted")
            name = place.get("properties", {}).get("name")
            distance = place.get("properties", {}).get("distance")
            elev = place.get("properties", {}).get("ele")
            website = place.get("properties", {}).get("website")
        
            # return object
            hotel_row = {
                "city": city,
                "country": country,
                "latitude": latitude,
                "longitude": longitude,
                "address": address,
                "name": name,
                "distance": distance,
                "elevation": elev,
                "website": website
            }
        else:
            hotel_row = {
                "city": city,
                "country": country,
                "latitude": latitude,
                "longitude": longitude,
                "address": None,
                "name": None,
                "distance": None,
                "elevation": None,
                "website": None
            }
    except Exception as e:
        print(e)
        hotel_row = {
                "city": city,
                "country": country,
                "latitude": latitude,
                "longitude": longitude,
                "address": None,
                "name": None,
                "distance": None,
                "elevation": None,
                "website": None
            }

    # append to hotel list
    hotel_rows.append(hotel_row)



### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [29]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    "longitude",
    "latitude",
    geo = True,
    tiles = "EsriNatGeo",
    frame_width = 700,
    frame_height = 500,
    color = "city",
    hover_cols=["city", "country", "name", "address"]  # Add 'city' to the tooltip
)

# Display the map plot
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [longitude,latitude]   (city,country,name,address)